# Projeto: Alocação de Aviões com OR-Tools

Este notebook demonstra o funcionamento de um modelo de alocação de aviões utilizando o solver CP-SAT da Google OR-Tools.

In [ ]:
!pip install ortools==9.7.2996 matplotlib --quiet

In [ ]:
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt

def alocar_avioes(total_avioes, total_estacionamentos, total_tempo,
                  momento_de_chegada, tempo_duracao, custos):
    modelo = cp_model.CpModel()
    solucionador = cp_model.CpSolver()

    X = [[[modelo.NewBoolVar(f'X_av{i}_est{j}_t{k}')
           for k in range(total_tempo)]
           for j in range(total_estacionamentos)]
           for i in range(total_avioes)]

    Y = [[modelo.NewBoolVar(f'Y_av{i}_est{j}')
           for j in range(total_estacionamentos)]
           for i in range(total_avioes)]

    for j in range(total_estacionamentos):
        for k in range(total_tempo):
            modelo.Add(sum(X[i][j][k] for i in range(total_avioes)) <= 1)

    for i in range(total_avioes):
        modelo.Add(sum(X[i][j][k] for j in range(total_estacionamentos)
                                      for k in range(total_tempo)) == tempo_duracao[i])
        modelo.Add(sum(Y[i]) == 1)

        for j in range(total_estacionamentos):
            uso = [X[i][j][k] for k in range(total_tempo)]
            modelo.Add(sum(uso) == tempo_duracao[i]).OnlyEnforceIf(Y[i][j])
            modelo.Add(sum(uso) == 0).OnlyEnforceIf(Y[i][j].Not())

            for k in range(momento_de_chegada[i]):
                modelo.Add(X[i][j][k] == 0)

        if momento_de_chegada[i] < total_tempo:
            modelo.Add(sum(X[i][j][momento_de_chegada[i]]
                           for j in range(total_estacionamentos)) == 1)

    modelo.Minimize(sum(custos[i][j] * Y[i][j]
                        for i in range(total_avioes)
                        for j in range(total_estacionamentos)))

    status = solucionador.Solve(modelo)

    resultados = []
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        for i in range(total_avioes):
            for j in range(total_estacionamentos):
                for k in range(total_tempo):
                    if solucionador.Value(X[i][j][k]):
                        resultados.append((i, j, k))
    return resultados

### Explicação da Função
Esta função monta o modelo de alocação com as variáveis X e Y, aplica restrições de ocupação, chegada, duração e minimiza o custo total.

## Exemplo Pequeno

In [ ]:
res = alocar_avioes(
    3, 2, 10,
    [0, 2, 5],
    [3, 4, 2],
    [
        [10, 50],
        [10, 60],
        [40, 5]
    ])
res

In [ ]:
def plot_resultados(res, total_avioes, total_estacionamentos, total_tempo):
    fig, ax = plt.subplots(figsize=(12, 6))
    cmap = plt.get_cmap("tab10")
    ocupacoes = {}

    for i, j, k in res:
        ocupacoes.setdefault((i, j), []).append(k)

    for (i, j), ks in ocupacoes.items():
        inicio = min(ks)
        fim = max(ks)
        ax.broken_barh([(inicio, fim - inicio + 1)], (i * 10, 9),
                       facecolors=cmap(j % 10), edgecolor='black')
        ax.text(inicio + (fim - inicio)/2, i * 10 + 4.5, f"E{j}", ha='center', va='center', color='white')

    ax.set_xlabel("Tempo")
    ax.set_ylabel("Aviões")
    ax.set_yticks([i * 10 + 4.5 for i in range(total_avioes)])
    ax.set_yticklabels([f"Avião {i}" for i in range(total_avioes)])
    ax.grid(True)
    plt.title("Alocação de Aviões")
    plt.tight_layout()
    plt.show()

plot_resultados(res, 3, 2, 10)

## Exemplo Grande
Neste exemplo, testamos a alocação de 12 aviões ao longo de 24 unidades de tempo, com 5 estacionamentos disponíveis.

### Resultado e Visualização
A saída da função `alocar_avioes` é uma lista de tuplas `(avião, estacionamento, tempo)`, indicando onde cada avião está alocado. A função `plot_resultados` gera um gráfico com essas alocações.

In [ ]:
res2 = alocar_avioes(
    12, 5, 24,
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    [3, 4, 5, 2, 3, 4, 2, 3, 2, 4, 3, 3],
    [[(j + 1) * 100 + i * 5 for j in range(5)] for i in range(12)]
)
res2

In [ ]:
plot_resultados(res2, 12, 5, 24)